显示逐时效产品生成时间


In [1]:
import pandas as pd
from nwpc_message_tool import nmc_monitor

engine = nmc_monitor


time_series = pd.date_range(
    start="2020-03-08T12:00:00",
    end="2020-03-18T23:00:00",
    freq="D",
)

start_time = list(s.to_pydatetime() for s in time_series)

system = "nwpc_grapes_gfs"
system = engine.fix_system_name(system)

elastic_server = "localhost:9200"

In [2]:
from nwpc_message_tool.storage import EsMessageStorage

client = EsMessageStorage(
    hosts=elastic_server,
    engine=engine,
)

results = client.get_production_messages(
    system=system,
    production_stream="oper",
    production_type="grib2",
    production_name="orig",
    start_time=start_time
)

results = list(results)

KeyboardInterrupt: 

In [ ]:
from nwpc_message_tool.processor import TableProcessor

processor = TableProcessor()
df = processor.process_messages(results)

In [ ]:
from nwpc_message_tool.presenter import StepGridPlotPresenter

presenter = StepGridPlotPresenter(
    system=system,
    output_type=("notebook",),
)
presenter.show(df)